In [7]:
import os
import base64
from openai import AzureOpenAI
from PIL import Image

#keys
keys_file = "azure_keys.txt"  # same location as this script

with open(keys_file, "r") as f:
    for line in f:
        if "=" in line:
            k, v = line.strip().split("=", 1)
            os.environ[k] = v

AZURE_OPENAI_API_VERSION = "2024-02-01"
GPT_DEPLOYMENT_NAME = "gpt-4o"


client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
)

#encodeimage
def encode_image(img_path):
    with open(img_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

##few shot examples
example_dir = r"C:\Users\94718\Downloads\CW\ANN-based-Acute-Lymphoblastic-Leukemia-Classifier\Data\GenAI"
classes = ["Benign", "Early", "Pre", "Pro"]

example_images = []
for label in classes:
    folder = os.path.join(example_dir, label)
    samples = sorted(os.listdir(folder))[:5]
    for img_file in samples:
        img_path = os.path.join(folder, img_file)
        example_images.append((label, encode_image(img_path)))

#query
query_path = r"C:\Users\94718\Downloads\CW\ANN-based-Acute-Lymphoblastic-Leukemia-Classifier\Data\Preprocessed\Early\WBC-Malignant-Early-007.jpg"
query_image = encode_image(query_path)

#base prompt
prompt = """
You are a medical AI assistant. This task is to detect Acute Lymphoblastic Leukemia with peripheral blood smear images.
We are classifying peripheral blood smear (PBS) images into: Benign, Early, Pre, Pro. 
Use the few-shot examples provided to guide classification. 
Cells can be described as:
1. Benign: normal and healthy  
2. Early: Pre-B cells, very early-stage cancerous cells, difficult to detect  
3. Pre: Pre-B cells, intermediate malignant cells, most common subtype  
4. Pro: Pro-B cells, immature and highly aggressive cancerous cells  

After classifying the query image, also generate an explanation as if you were producing a Grad-CAM heatmap:
- Highlight which regions of the image the model would focus on
- Explain why the model predicted this class.
"""

#build message
messages = [{"role": "system", "content": "You are a helpful AI for cancer image classification."}]

# Add few-shot examples
for label, img_b64 in example_images:
    messages.append({
        "role": "user",
        "content": [
            {"type": "text", "text": f"Example image of class: {label}"},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}}
        ],
    })
    messages.append({
        "role": "assistant",
        "content": f"This is a {label} sample image."
    })

# Add the query image
messages.append({
    "role": "user",
    "content": [
        {"type": "text", "text": prompt},
        {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{query_image}"}}
    ]
})

#gpt call
response = client.chat.completions.create(
    model=GPT_DEPLOYMENT_NAME,
    messages=messages
)

print(response.choices[0].message.content)


### Classification: Early

### Explanation:

#### Grad-CAM Heatmap Focus:
The model would focus primarily on the clustered purple-stained cells seen in the image. Specifically, the model would highlight the five central cells that show distinctive morphological characteristics. 

#### Reason for Prediction:

1. **Cell Size and Shape**: The highlighted cells exhibit a relatively uniform and small size, which is indicative of Early-stage cancerous cells (Pre-B cells). This uniformity aligns with the characteristics of Early-stage cells, which tend to be more consistent in size compared to later-stage malignant cells. 

2. **Nuclear Features**: The nuclei of these cells are well-defined and smooth, which is typical for Early-stage malignant cells. The purple staining is more intense, suggesting active cellular processes, but the nuclei are not as irregular or significantly enlarged, which differentiates them from the more aggressive ‘Pre’ and ‘Pro’ stages.

3. **Background Cell Density**:

In [8]:
import os
import base64
import pandas as pd
from openai import AzureOpenAI
from PIL import Image

# -------------------------
# 1. Load Azure keys
# -------------------------
keys_file = "azure_keys.txt"
with open(keys_file, "r") as f:
    for line in f:
        if "=" in line:
            k, v = line.strip().split("=", 1)
            os.environ[k] = v

AZURE_OPENAI_API_VERSION = "2024-02-01"
GPT_DEPLOYMENT_NAME = "gpt-4o"

client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
)

# -------------------------
# 2. Helper: encode image
# -------------------------
def encode_image(img_path):
    with open(img_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

# -------------------------
# 3. Few-shot examples
# -------------------------
example_dir = r"C:\Users\94718\Downloads\CW\ANN-based-Acute-Lymphoblastic-Leukemia-Classifier\Data\GenAI"
classes = ["Benign", "Early", "Pre", "Pro"]

example_images = []
for label in classes:
    folder = os.path.join(example_dir, label)
    samples = sorted(os.listdir(folder))[:5]
    for img_file in samples:
        img_path = os.path.join(folder, img_file)
        example_images.append((label, encode_image(img_path)))

# -------------------------
# 4. Base prompt for classification
# -------------------------
base_prompt = """
You are a medical AI assistant. This task is to detect Acute Lymphoblastic Leukemia with peripheral blood smear images.
We are classifying peripheral blood smear (PBS) images into: Benign, Early, Pre, Pro. 
Use the few-shot examples provided to guide classification. 
Cells can be described as:
1. Benign: normal and healthy  
2. Early: Pre-B cells, very early-stage cancerous cells, difficult to detect  
3. Pre: Pre-B cells, intermediate malignant cells, most common subtype  
4. Pro: Pro-B cells, immature and highly aggressive cancerous cells  

After classifying the query image, also generate an explanation as if you were producing a Grad-CAM heatmap:
- Highlight which regions of the image the model would focus on
- Explain why the model predicted this class.
"""

# -------------------------
# 5. Folder with images to test
# -------------------------
test_dir = r"C:\Users\94718\Downloads\CW\ANN-based-Acute-Lymphoblastic-Leukemia-Classifier\Data\genai-test"

results = []

# Iterate over each subfolder (class)
for true_class in os.listdir(test_dir):
    folder_path = os.path.join(test_dir, true_class)
    if not os.path.isdir(folder_path):
        continue
    
    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        img_b64 = encode_image(img_path)
        
        # -------------------------
        # 6. Build messages
        # -------------------------
        messages = [{"role": "system", "content": "You are a helpful AI for cancer image classification."}]
        
        # Add few-shot examples
        for label, ex_b64 in example_images:
            messages.append({
                "role": "user",
                "content": [
                    {"type": "text", "text": f"Example image of class: {label}"},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{ex_b64}"}}
                ]
            })
            messages.append({
                "role": "assistant",
                "content": f"This is a {label} sample image."
            })
        
        # Add query image
        messages.append({
            "role": "user",
            "content": [
                {"type": "text", "text": base_prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}}
            ]
        })
        
        # -------------------------
        # 7. GPT Call
        # -------------------------
        response = client.chat.completions.create(
            model=GPT_DEPLOYMENT_NAME,
            messages=messages
        )
        
        prediction_text = response.choices[0].message.content
        
        # Store results
        results.append({
            "Image": img_file,
            "True_Class": true_class,
            "Predicted_Output": prediction_text
        })
        print(f"Processed {img_file} - True: {true_class}")


# save results
results_df = pd.DataFrame(results)
results_df.to_csv("test_results.csv", index=False)
print("Results saved to test_results.csv")


Processed benign_aug_33.jpg - True: Benign
Processed benign_aug_34.jpg - True: Benign
Processed WBC-Malignant-Early-035.jpg - True: Early
Processed WBC-Malignant-Early-036.jpg - True: Early
Processed WBC-Malignant-Pre-036.jpg - True: Pre
Processed WBC-Malignant-Pre-037.jpg - True: Pre
Processed WBC-Malignant-Pro-038.jpg - True: Pro
Processed WBC-Malignant-Pro-039.jpg - True: Pro
Results saved to test_results.csv
